In [3]:
import pandas as pd
from keras.layers import Dense, Dropout, Activation, Input
from keras.models import Model

In [4]:
base = pd.read_csv('games.csv')

In [5]:
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [5]:
base.shape

(16719, 13)

In [6]:
base = base.dropna(axis=0)
base.shape

(6825, 13)

In [7]:
base = base.loc[base['NA_Sales'] > 1]
base.shape

(584, 13)

In [8]:
base = base.loc[base['EU_Sales'] > 1]
base.shape

(258, 13)

In [9]:
base['Name'].value_counts()

Name
Grand Theft Auto V                 4
Assassin's Creed IV: Black Flag    3
Call of Duty: Ghosts               3
Battlefield 4                      3
The Elder Scrolls V: Skyrim        3
                                  ..
Tekken 2                           1
Destiny                            1
Cooking Mama                       1
Forza Motorsport 3                 1
The Sims: Bustin' Out              1
Name: count, Length: 223, dtype: int64

In [10]:
games_names = base.Name
base = base.drop('Name', axis=1)

In [11]:
predictors = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values

In [12]:
sales_na = base.iloc[:, 4].values
sales_eu = base.iloc[:, 5].values
sales_jp = base.iloc[:, 6].values

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ("c0", OneHotEncoder(), [0]),
    ("c2", OneHotEncoder(), [2]),
    ("c3", OneHotEncoder(), [3]),
    ("c8", OneHotEncoder(), [8])
    
    ], remainder='passthrough')

In [14]:
predictors = ct.fit_transform(predictors).toarray()
predictors[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 2.006e+03, 7.600e+01, 5.100e+01, 8.000e+00,
       3.220e+02])

In [15]:
predictors.shape

(258, 61)

In [16]:
entry_layer = Input(shape=(61,))
hidden1_layer = Dense(units=32, activation='sigmoid')(entry_layer)
hidden2_layer = Dense(units=32, activation='sigmoid')(hidden1_layer)
output1_layer = Dense(units=1, activation='linear')(hidden2_layer)
output2_layer = Dense(units=1, activation='linear')(hidden2_layer)
output3_layer = Dense(units=1, activation='linear')(hidden2_layer)

In [17]:
regressor = Model(inputs = entry_layer, outputs=[output1_layer, output2_layer, output3_layer])

In [18]:
regressor.compile(optimizer='adam', loss='mse')

In [ ]:
regressor.fit(predictors, [sales_na, sales_eu, sales_jp], epochs=5000, batch_size=100)

Epoch 1/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - loss: 40.0760
Epoch 2/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.2752 
Epoch 3/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.0696 
Epoch 4/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 28.5152 
Epoch 5/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31.0061 
Epoch 6/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 31.1881 
Epoch 7/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 34.1437 
Epoch 8/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 28.3033 
Epoch 9/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 23.5752 
Epoch 10/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 26.7544 
Epoch 11/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 26.1820 
Epoch 12/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.5360 
Epoch 13/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 30.8324 
Epoch 14/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 29.3638 
Epoch 15/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss

In [20]:
prediction_na, prediction_eu, prediction_jp = regressor.predict(predictors)

9/9 [==============================] - 0s 1ms/step


In [21]:
sales_na[0:10]

array([41.36, 15.68, 15.61, 11.28, 13.96, 14.44,  9.71,  8.92, 15.  ,
        9.01])

In [22]:
prediction_na[0:10]

array([[13.745984 ],
       [14.813007 ],
       [13.781742 ],
       [ 9.123977 ],
       [13.566464 ],
       [11.600856 ],
       [10.630115 ],
       [12.300923 ],
       [10.297885 ],
       [ 3.7286854]], dtype=float32)